In [1]:
repo_dir = "Repos"  
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-vgg19" / "src" / "vgg19"))
sys.path.insert(0, str(Path.home() / repo_dir / "RETFound_MAE"))
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-exec" / "models" / "vgg19")) 

In [2]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration, DatasetVersion
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

2025-03-30 13:06:34.856127: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-30 13:06:34.856189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-30 13:06:34.996246: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-30 13:06:35.308006: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-30 13:06:37.055041: W tensorflow/compiler/tf2

In [3]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

You are already logged in.


In [4]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-03-30 13:06:42,447 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install


In [5]:
datasets = [
    '4-4116', # Selected images for training
    '4-411G', # Selected images for testing
    # '2-7P5P', # Full multimodal dataset
    ]

to_be_download = []
for dataset in datasets:
    ds_dict = {
        'rid': dataset,
        'materialize':True,
        'version':EA.dataset_version(dataset_rid=dataset),
    }
    to_be_download.append(ds_dict)

workflow_instance = EA.create_workflow(
    name="Multimodal workflow",
    workflow_type="Multimodal workflow"
)

config = ExecutionConfiguration(
    datasets=to_be_download,
    assets = ['2-4JR6',],
    workflow=workflow_instance,
    description="Instance of cropping multimodal images.")

execution = EA.create_execution(config)

2025-03-30 13:08:31,667 - INFO - Materialize bag 4-4116... 
2025-03-30 13:08:32,604 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install
2025-03-30 13:08:33,928 - INFO - Loading /data/4-4116_5a611f3561ac538bae3767ae4c1bafa1533b9722fca8a1fc78085f422c5290f6/Dataset_4-4116
2025-03-30 13:08:34,714 - INFO - Creating new database for dataset: 4-4116 in /data/nguyent8/EyeAI_working/4-4116@330-B1Z0-2NHT.db
2025-03-30 13:08:34,716 - INFO - Materialize bag 4-411G... 
2025-03-30 13:08:35,455 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install
2025-03-30 13:08:36,284 - INFO - Loading /data/4-411G_d5fd9f998a7b713e17b9b70eec4cef9d70f7b3bfd0a0cd3931d4e2edbade797d/Dataset_4-411G
2025-03-30 13:08:36,815 - INFO - Creating new database for dataset: 4-411G in /data/nguyent8/EyeAI_working/4-411G@330-B1Z0-GC2Y.db
2025-03-30 13:08:36,877 - INFO - Downloading assets ...
2025-03-30 13:08:40,744 - INFO - File [/data/nguyent8/EyeAI_working/4-S5

In [6]:
print(execution)

caching_dir: /data
_working_dir: /data/nguyent8/EyeAI_working
execution_rid: 4-S51E
workflow_rid: 4-S51C
asset_paths: [PosixPath('/data/nguyent8/EyeAI_working/4-S51E/asset/optic_disk_crop_model.hdf5')]
configuration: datasets=[DatasetSpec(rid='4-4116', materialize=True, version=DatasetVersion(major=2, minor=1, patch=0)), DatasetSpec(rid='4-411G', materialize=True, version=DatasetVersion(major=2, minor=1, patch=0))] assets=['2-4JR6'] workflow=Workflow(name='Multimodal workflow', url='https://github.com/informatics-isi-edu/eye-ai-exec/blob//notebooks/crop_multimodal_images.ipynb', workflow_type='Multimodal workflow', version=None, description='', rid=None, checksum='e69de29bb2d1d6434b8b29ae775ad8c2e48c5391') parameters={} description='Instance of cropping multimodal images.' argv=['/home/nguyent8/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/home/nguyent8/.local/share/jupyter/runtime/kernel-58a74d22-42b7-4b11-866b-8ca10afabfa0.json']


In [ ]:
training_ds_bag = execution.datasets[0]
testing_ds_bag = execution.datasets[1]
# multimodal_full_ds_bag = execution.datasets[2]

crop_image_model = execution.asset_paths[0]